# MNIST Classification: Hyperparameter Comparison

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt

configs = [
    {"activation": nn.Sigmoid(), "epochs": 24, "val_split": 0.3, "opt": optim.Adagrad, "lr": 0.001},
    {"activation": nn.ReLU(), "epochs": 25, "val_split": 0.2, "opt": optim.Adam, "lr": 0.001},
    {"activation": nn.Tanh(), "epochs": 21, "val_split": 0.1, "opt": optim.Adadelta, "lr": 0.001},
    {"activation": nn.Sigmoid(), "epochs": 23, "val_split": 0.2, "opt": optim.Adam, "lr": 0.001},
]

class MLP(nn.Module):
    def __init__(self, act):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 256),
            act,
            nn.Linear(256, 128),
            act,
            nn.Linear(128, 10)
        )
    def forward(self, x): return self.model(x)

transform = transforms.ToTensor()
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

for i, cfg in enumerate(configs, 1):
    val_len = int(cfg["val_split"] * len(mnist))
    train_len = len(mnist) - val_len
    train_set, val_set = random_split(mnist, [train_len, val_len])
    train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=64)

    model = MLP(cfg["activation"])
    loss_fn = nn.CrossEntropyLoss()
    optimizer = cfg["opt"](model.parameters(), lr=cfg["lr"])

    print(f"Scenario {i}:")
    for epoch in range(cfg["epochs"]):
        model.train()
        for imgs, labels in train_loader:
            preds = model(imgs)
            loss = loss_fn(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{cfg['epochs']}, Loss={loss.item():.4f}")
